### 문제 2-1: 콤마 구분 리스트 파서 활용

In [1]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 콤마로 구분된 리스트 출력 파서 초기화
output_parser = CommaSeparatedListOutputParser()

# 출력 형식 지침 가져오기
format_instructions = output_parser.get_format_instructions()

# 프롬프트 템플릿 설정
prompt = PromptTemplate(
    template="사용자의 관심분야인 {subject}와 관련된 한국의 유명한 장소나 활동 5가지를 추천해주세요.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},
)

# 언어 모델 설정
llm = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0.7
)

# 체인 생성
chain = prompt | llm | output_parser

# 체인 호출 및 결과 출력
subject = "음식"
result = chain.invoke({"subject": subject})
print(f"'{subject}' 관련 추천: {result}")

subject = "스포츠"
result = chain.invoke({"subject": subject})
print(f"'{subject}' 관련 추천: {result}")

subject = "영화"
result = chain.invoke({"subject": subject})
print(f"'{subject}' 관련 추천: {result}")

'음식' 관련 추천: ['광장시장 먹자골목', '서울 야시장', '전주한옥마을 먹거리', '제주도 흑돼지 맛집 투어', '김장체험']
'스포츠' 관련 추천: ['잠실종합운동장', '서울올림픽공원', '태릉선수촌', '한강공원 자전거길', '스키월드 랜드프로그램']
'영화' 관련 추천: ['부산국제영화제', '춘천애니메이션박물관', '서울영화촬영소', 'CGV용산아이파크몰아이맥스', '한국영화자료관']


### 문제 2-2: 영화 리뷰 감정 분석기

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import EnumOutputParser
from enum import Enum

# 감정 클래스 정의
class Sentiment(str, Enum):
    POSITIVE = "긍정"
    NEGATIVE = "부정"
    NEUTRAL = "보통"

# EnumOutputParser 초기화
parser = EnumOutputParser(enum=Sentiment)

# 프롬프트 템플릿
template = """
당신은 텍스트 감정 분석 전문가입니다.
다음 텍스트의 감정을 분석하고, 반드시 아래 세 가지 중 하나의 단어로만 답변하세요.

텍스트: "{text}"

{format_instructions}

중요 규칙:
1. 반드시 "긍정", "부정", "보통" 중 하나의 단어만 출력하세요.
2. 다른 설명이나 부가 설명을 추가하지 마세요.
"""

prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=parser.get_format_instructions())

# 언어 모델 설정
llm = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0
)

# 체인 생성
chain = prompt | llm | parser

# 테스트 리뷰
test_reviews = [
    "이 영화 정말 재미없어요. 시간 낭비였습니다.",
    "배우들의 연기가 훌륭하고 스토리도 감동적이었어요!",
    "그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요."
]

# 리뷰 감정 분석 및 출력
for review in test_reviews:
    result = chain.invoke({"text": review})
    print(f"리뷰: \"{review}\" -> 감정: {result.value}")

리뷰: "이 영화 정말 재미없어요. 시간 낭비였습니다." -> 감정: 부정
리뷰: "배우들의 연기가 훌륭하고 스토리도 감동적이었어요!" -> 감정: 긍정
리뷰: "그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요." -> 감정: 보통


### 문제 2-3: 학생 정보 구조화 시스템

In [4]:
import json
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List

# 출력 구조를 정의하는 Pydantic 모델
class StudentInfo(BaseModel):
    name: str = Field(description="학생의 이름")
    age: int = Field(description="학생의 나이")
    major: str = Field(description="학생의 전공")
    hobbies: List[str] = Field(description="학생의 취미 리스트")
    goal: str = Field(description="학생의 목표")

# Pydantic 출력 파서 초기화
parser = PydanticOutputParser(pydantic_object=StudentInfo)

# 프롬프트 템플릿 설정
template = """
학생의 자기소개를 분석하여 아래 형식에 맞게 정보를 추출해주세요.
자기소개: {introduction}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=parser.get_format_instructions())

# 언어 모델 설정
llm = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0
)

# 체인 생성
chain = prompt | llm | parser

# 예시 입력
introduction = "안녕하세요! 저는 김민수이고 22살입니다. 컴퓨터공학을 전공하고 있어요. 취미로는 게임하기, 영화보기, 코딩을 좋아합니다. 앞으로 훌륭한 개발자가 되는 것이 목표입니다."

# 정보 추출
result = chain.invoke({"introduction": introduction})

# JSON 형식으로 예쁘게 출력
print(json.dumps(result.dict(), indent=4, ensure_ascii=False))

{
    "name": "김민수",
    "age": 22,
    "major": "컴퓨터공학",
    "hobbies": [
        "게임하기",
        "영화보기",
        "코딩"
    ],
    "goal": "훌륭한 개발자가 되는 것"
}


C:\Users\user\AppData\Local\Temp\ipykernel_6920\1973774720.py:47: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  print(json.dumps(result.dict(), indent=4, ensure_ascii=False))


### 문제 2-4: 여행 계획 분석기

In [5]:
import json
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

# 출력 구조 정의
response_schemas = [
    ResponseSchema(name="destination", description="여행지"),
    ResponseSchema(name="duration", description="여행 기간"),
    ResponseSchema(name="budget", description="총 예산"),
    ResponseSchema(name="rating", description="추천도 (1-5점)"),
    ResponseSchema(name="activities", description="주요 활동 리스트", type="list")
]

# 파서 초기화
parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = parser.get_format_instructions()

# 프롬프트 템플릿
template = """
다음 여행 후기 텍스트를 분석하여 아래 형식에 맞게 정보를 추출하세요.
후기: {review}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=format_instructions)

# 언어 모델 설정
llm = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0
)

# 체인 생성
chain = prompt | llm | parser

# 예시 입력
review = "지난 주에 부산으로 2박 3일 여행을 다녀왔어요. 총 30만원 정도 썼는데 해운대에서 바다구경하고, 자갈치시장에서 회 먹고, 감천문화마을도 구경했어요. 정말 만족스러운 여행이었습니다. 5점 만점에 4점 정도 줄 수 있을 것 같아요."

# 정보 추출
result = chain.invoke({"review": review})

# JSON 형식으로 예쁘게 출력
print(json.dumps(result, indent=4, ensure_ascii=False))

{
    "destination": "부산",
    "duration": "2박 3일",
    "budget": "30만원",
    "rating": "4점",
    "activities": [
        "해운대 바다구경",
        "자갈치시장 회 먹기",
        "감천문화마을 구경"
    ]
}
